<a href="https://colab.research.google.com/github/supriyamk2/Machine-learning/blob/main/covid_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import glob
import json
import time
from tqdm import tqdm

In [10]:
import pickle5 as pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Accessing the research papers and data

with open('/content/drive/MyDrive/datasets/metadata_sample.pkl', 'rb') as f:

  df_metadata = pickle.load(f)
with open('/content/drive/MyDrive/datasets/json_pdf_sample.pkl', 'rb') as f:
  df_pdf = pickle.load(f)
with open('/content/drive/MyDrive/datasets/json_pmc_sample.pkl', 'rb') as f:
  df_pmc = pickle.load(f)

In [15]:
df_metadata.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
600880,n441sprx,feedfe27a4eee49d8a1d09f50e8ecfe73057602a,Medline; PMC; WHO,Experiences and effects of telerehabilitation ...,10.4102/sajp.v77i1.1528,PMC8252170,34230898.0,cc-by,BACKGROUND: The announcement of a national loc...,2021-06-30,"Ebrahim, Humairaa; Pillay-Jayaraman, Prithi; L...",S Afr J Physiother,NaN,NaN,NaN,document_parses/pdf_json/feedfe27a4eee49d8a1d0...,document_parses/pmc_json/PMC8252170.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/34230898/;...,235720984.0
721427,95libakm,c2ca01f12643a88e059e81619fb971fa61de3971,Elsevier; Medline; PMC,SCOAT-Net: A novel network for segmenting COVI...,10.1016/j.patcog.2021.108109,PMC8189738,34127870.0,no-cc,Automatic segmentation of lung opacification f...,2021-06-10,"Zhao, Shixuan; Li, Zhidan; Chen, Yang; Zhao, W...",Pattern Recognit,NaN,NaN,NaN,document_parses/pdf_json/c2ca01f12643a88e059e8...,document_parses/pmc_json/PMC8189738.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/34127870/;...,235382189.0
701284,bjl0bjg2,d0d1e05bb14f068d8323807ba042860887b7aa00,Medline; PMC,A Comprehensive Review on Factors Influences B...,10.2147/ijn.s291956,PMC7898217,33628021.0,cc-by-nc,Exosomes are nanoscale-sized membrane vesicles...,2021-02-17,"Gurunathan, Sangiliyandi; Kang, Min-Hee; Kim, ...",Int J Nanomedicine,NaN,NaN,NaN,document_parses/pdf_json/d0d1e05bb14f068d83238...,document_parses/pmc_json/PMC7898217.xml.json,https://doi.org/10.2147/ijn.s291956; https://w...,232016676.0
603427,7ch1yln4,fb8c3a315b4a345dafc7e59be000e21bf965dea8,Medline; PMC,Opposition to cannabis legalization on public ...,10.1016/j.lanwpc.2021.100142,PMC8315435,34327442.0,cc-by-nc-nd,NaN,2021-04-13,"Smyth, Bobby P; Christie, Grant IG",Lancet Reg Health West Pac,NaN,NaN,NaN,document_parses/pdf_json/fb8c3a315b4a345dafc7e...,document_parses/pmc_json/PMC8315435.xml.json,https://doi.org/10.1016/j.lanwpc.2021.100142; ...,234824471.0
771175,zegp4jq4,ed4c180c272b047c6db958508fb9a8edd40cb1cb,Elsevier; Medline; PMC,An epigenetic signature to fight COVID-19,10.1016/j.ebiom.2021.103385,PMC8116817,33993054.0,no-cc,NaN,2021-05-13,"Herbein, Georges",EBioMedicine,NaN,NaN,NaN,document_parses/pdf_json/ed4c180c272b047c6db95...,document_parses/pmc_json/PMC8116817.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33993054/;...,234487502.0


In [16]:
#Merging the metadata and json pdf data
#sha column of metadata and paper_id of json_pdf data are same identifier for each record
df_merged = pd.merge(df_metadata,df_pdf,left_on='sha',right_on='paper_id',how='left').drop('paper_id',axis=1)

In [17]:
df_merged.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract_x,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,abstract_y,body_text
0,n441sprx,feedfe27a4eee49d8a1d09f50e8ecfe73057602a,Medline; PMC; WHO,Experiences and effects of telerehabilitation ...,10.4102/sajp.v77i1.1528,PMC8252170,34230898.0,cc-by,BACKGROUND: The announcement of a national loc...,2021-06-30,"Ebrahim, Humairaa; Pillay-Jayaraman, Prithi; L...",S Afr J Physiother,NaN,NaN,NaN,document_parses/pdf_json/feedfe27a4eee49d8a1d0...,document_parses/pmc_json/PMC8252170.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/34230898/;...,235720984.0,,"On Thursday, 05 March 2020, the National Insti..."
1,95libakm,c2ca01f12643a88e059e81619fb971fa61de3971,Elsevier; Medline; PMC,SCOAT-Net: A novel network for segmenting COVI...,10.1016/j.patcog.2021.108109,PMC8189738,34127870.0,no-cc,Automatic segmentation of lung opacification f...,2021-06-10,"Zhao, Shixuan; Li, Zhidan; Chen, Yang; Zhao, W...",Pattern Recognit,NaN,NaN,NaN,document_parses/pdf_json/c2ca01f12643a88e059e8...,document_parses/pmc_json/PMC8189738.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/34127870/;...,235382189.0,• We propose a novel spatial-and channel-wise ...,"The coronavirus disease 2019 (COVID- 19) , whi..."
2,bjl0bjg2,d0d1e05bb14f068d8323807ba042860887b7aa00,Medline; PMC,A Comprehensive Review on Factors Influences B...,10.2147/ijn.s291956,PMC7898217,33628021.0,cc-by-nc,Exosomes are nanoscale-sized membrane vesicles...,2021-02-17,"Gurunathan, Sangiliyandi; Kang, Min-Hee; Kim, ...",Int J Nanomedicine,NaN,NaN,NaN,document_parses/pdf_json/d0d1e05bb14f068d83238...,document_parses/pmc_json/PMC7898217.xml.json,https://doi.org/10.2147/ijn.s291956; https://w...,232016676.0,Exosomes are nanoscale-sized membrane vesicles...,Extracellular vesicles (EVs) including exosome...
3,7ch1yln4,fb8c3a315b4a345dafc7e59be000e21bf965dea8,Medline; PMC,Opposition to cannabis legalization on public ...,10.1016/j.lanwpc.2021.100142,PMC8315435,34327442.0,cc-by-nc-nd,NaN,2021-04-13,"Smyth, Bobby P; Christie, Grant IG",Lancet Reg Health West Pac,NaN,NaN,NaN,document_parses/pdf_json/fb8c3a315b4a345dafc7e...,document_parses/pmc_json/PMC8315435.xml.json,https://doi.org/10.1016/j.lanwpc.2021.100142; ...,234824471.0,,In their commentary on the decision of New Zea...
4,zegp4jq4,ed4c180c272b047c6db958508fb9a8edd40cb1cb,Elsevier; Medline; PMC,An epigenetic signature to fight COVID-19,10.1016/j.ebiom.2021.103385,PMC8116817,33993054.0,no-cc,NaN,2021-05-13,"Herbein, Georges",EBioMedicine,NaN,NaN,NaN,document_parses/pdf_json/ed4c180c272b047c6db95...,document_parses/pmc_json/PMC8116817.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33993054/;...,234487502.0,,"A novel coronavirus, SARS-CoV-2, emerged in Ch..."


In [18]:
#Lets merge the json_pmc data to the merged data too
df_merged = pd.merge(df_merged,df_pmc,left_on='pmcid',right_on='paper_id',how='left').drop('paper_id',axis=1)
df_merged.shape

(25000, 22)

Data Cleaning and Preprocessing

In [19]:
df_merged[df_merged.abstract_x != df_merged.abstract_y].shape

(22724, 22)

In [20]:
# check metadata abstract column to see if null values exist
df_merged.abstract_x.isnull().sum(),(df_merged.abstract_x == '').sum()

(3597, 0)

In [21]:
# Check pdf_json abstract to see if null values exist
df_merged.abstract_y.isnull().sum(),(df_merged.abstract_y == '').sum()

(1525, 6902)

In [22]:
# Convert all columns to string and then replace abstract_y values
#df = df.astype(str)
df_merged["abstract_y"] = df_merged["abstract_y"].astype(str) 
df_merged['abstract_y'] = np.where(df_merged['abstract_y'].map(len) > 50, df_merged['abstract_y'], "na")

In [23]:
df_merged[df_merged['abstract_y'].apply(lambda x: len(str(x)) <= 10)]


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract_x,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,abstract_y,body_text_x,body_text_y
0,n441sprx,feedfe27a4eee49d8a1d09f50e8ecfe73057602a,Medline; PMC; WHO,Experiences and effects of telerehabilitation ...,10.4102/sajp.v77i1.1528,PMC8252170,34230898.0,cc-by,BACKGROUND: The announcement of a national loc...,2021-06-30,"Ebrahim, Humairaa; Pillay-Jayaraman, Prithi; L...",S Afr J Physiother,NaN,NaN,NaN,document_parses/pdf_json/feedfe27a4eee49d8a1d0...,document_parses/pmc_json/PMC8252170.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/34230898/;...,235720984.0,na,"On Thursday, 05 March 2020, the National Insti...","As a result of the COVID-19 pandemic, many pat..."
3,7ch1yln4,fb8c3a315b4a345dafc7e59be000e21bf965dea8,Medline; PMC,Opposition to cannabis legalization on public ...,10.1016/j.lanwpc.2021.100142,PMC8315435,34327442.0,cc-by-nc-nd,NaN,2021-04-13,"Smyth, Bobby P; Christie, Grant IG",Lancet Reg Health West Pac,NaN,NaN,NaN,document_parses/pdf_json/fb8c3a315b4a345dafc7e...,document_parses/pmc_json/PMC8315435.xml.json,https://doi.org/10.1016/j.lanwpc.2021.100142; ...,234824471.0,na,In their commentary on the decision of New Zea...,Both authors contributed to the drafting of th...
4,zegp4jq4,ed4c180c272b047c6db958508fb9a8edd40cb1cb,Elsevier; Medline; PMC,An epigenetic signature to fight COVID-19,10.1016/j.ebiom.2021.103385,PMC8116817,33993054.0,no-cc,NaN,2021-05-13,"Herbein, Georges",EBioMedicine,NaN,NaN,NaN,document_parses/pdf_json/ed4c180c272b047c6db95...,document_parses/pmc_json/PMC8116817.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33993054/;...,234487502.0,na,"A novel coronavirus, SARS-CoV-2, emerged in Ch...",Research performed in our laboratories is supp...
5,2rnllfz8,33703e2029e591656f0292aada7918ca734c4688,PMC,Alpha-1 Antitrypsin as a Therapeutic Agent for...,10.1007/978-3-319-23449-6_8,PMC7121596,NaN,no-cc,Alpha-1 antitrypsin is a positive acute phase ...,2015-07-23,"Wanner, Adam",Alpha-1 Antitrypsin,NaN,NaN,NaN,document_parses/pdf_json/33703e2029e591656f029...,document_parses/pmc_json/PMC7121596.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,na,"For many years, alpha-1 antitrypsin has been u...","For many years, alpha-1 antitrypsin has been u..."
6,tmhpbof5,bd1a86a6c01513ed0b1cf8fc1590cf5962111f98,PMC,Antibodies induced by receptor-binding domain ...,10.1016/j.jinf.2013.05.002,PMC7127489,23685240,no-cc,NaN,2013-05-16,"Du, Lanying; Ma, Cuiqing; Jiang, Shibo",J Infect,NaN,NaN,NaN,document_parses/pdf_json/bd1a86a6c01513ed0b1cf...,document_parses/pmc_json/PMC7127489.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,na,Antibodies induced by receptorbinding domain i...,No reported conflicts.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24986,d30hc3nb,00f864f7ee6d7aa005954da9c45f76515925e8f3,Elsevier; Medline; PMC,Inactivated COVID-19 vaccines to make a global...,10.1016/s1473-3099(21)00020-7,PMC7906657,33548196.0,no-cc,NaN,2021-02-03,"Iversen, Patrick L; Bavari, Sina",Lancet Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/00f864f7ee6d7aa005954...,document_parses/pmc_json/PMC7906657.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33548196/;...,231866464.0,na,Given that the risk of more severe COVID-19 in...,Many inactivated vaccines against severe acute...
24987,pzlkvyjk,a99927909867bf30530558770197e81e8efe398c,Medline; PMC,SARS-CoV-2 in the kidney: bystander or culprit?,10.1038/s41581-020-00354-7,PMC7487445,32929200.0,no-cc,A new study examined post-mortem kidney tissue...,2020-09-14,"Vijayan, Anitha; Humphreys, Benjamin D.",Nat Rev Nephrol,NaN,NaN,NaN,document_parses/pdf_json/a99927909867bf3053055...,document_parses/pmc_json/PMC7487445.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32929200/;...,221643861.0,na,Severe acute respiratory syndrome corona virus...,"Refers to Braun, F. et al. SARS-CoV-2 renal tr..."
24990,cflbdc15,3e014d18b1097

In [24]:
# replace abstract_x (metadata column) with abstract_y (pdf_json) value where abstract_x is null
df_merged.loc[df_merged.abstract_x.isnull() & (df_merged.abstract_y != 'na'),'abstract_x'] = df_merged[df_merged.abstract_x.isnull() & (df_merged.abstract_y != 'na')].abstract_y

In [25]:
# Do we have any remaining null abstract values. Not anymore. This is good.
# The null values have reduced which is what we had expected.
df_merged.abstract_x.isnull().sum()

2951

In [26]:
# Lets get rid of the pdf_json abstract column and rename the metadata abstract column
df_merged.rename(columns = {'abstract_x' : 'abstract'}, inplace = True)
df_merged.drop('abstract_y',axis=1,inplace = True)
df_merged.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id', 'body_text_x', 'body_text_y'],
      dtype='object')

In [27]:
# This is expected because body text comes from pdf and pmc folders
(df_merged.body_text_x != df_merged.body_text_y).sum()

25000

In [28]:
df_merged.body_text_x.isnull().sum(),(df_merged.body_text_y == '').sum()

(1525, 0)

In [29]:
# This is expected because there are less papers in json_pmc
df_merged.body_text_y.isnull().sum()

5393

In [30]:
# body_text_x is pdf_json. body_text_y comes from pmc_json
# Where available we use the text from pmc file trusting the statement quality
df_merged.body_text_x.isnull().sum(),(df_merged.body_text_y.isnull()).sum()

(1525, 5393)

In [31]:
# There are ~13k rows where body_text_x is null but body_text_y is not null
df_merged.loc[df_merged.body_text_x.isnull() & df_merged.body_text_y.notnull()]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,body_text_x,body_text_y
40,j1sefamd,de4ddeb6cab0c5e13f3a6543ad5b7bb1215dc1da; 8e92...,Medline; PMC,The effect of eviction moratoria on the transm...,10.1038/s41467-021-22521-5,PMC8050248,33859196.0,cc-by,Massive unemployment during the COVID-19 pande...,2021-04-15,"Nande, Anjalika; Sheen, Justin; Walters, Emma ...",Nat Commun,NaN,NaN,NaN,document_parses/pdf_json/de4ddeb6cab0c5e13f3a6...,document_parses/pmc_json/PMC8050248.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33859196/;...,233257667.0,NaN,The COVID-19 epidemic has caused unprecedented...
76,x1mb8so9,309cc336df9d42f9189c9e5b2221e177bdacc89b; 9604...,Medline; PMC,"Early Transmission Dynamics, Spread, and Genom...",10.3201/eid2702.203767,PMC7853578,33496228.0,no-cc,"We report an epidemiologic analysis of 4,210 c...",2021-02-19,"Franco, Danilo; Gonzalez, Claudia; Abrego, Ley...",Emerg Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/309cc336df9d42f9189c9...,document_parses/pmc_json/PMC7853578.xml.json,https://doi.org/10.3201/eid2702.203767; https:...,231711065.0,NaN,"To perform molecular detection of SARS-CoV-2, ..."
83,v9xo446h,08a3e8af143f9b97a9ba5220f0de0fe0ae1bba06; 15d4...,Elsevier; Medline; PMC,Regulation of stress granules in virus systems,10.1016/j.tim.2012.02.001,PMC3322245,22405519.0,green-oa,Virus infection initiates a number of cellular...,2012-04-01,"White, James P.; Lloyd, Richard E.",Trends in Microbiology,NaN,NaN,NaN,document_parses/pdf_json/08a3e8af143f9b97a9ba5...,document_parses/pmc_json/PMC3322245.xml.json,https://doi.org/10.1016/j.tim.2012.02.001; htt...,46260378.0,NaN,Eukaryotic cells can contain multiple types of...
86,39rck23u,480c6748788911bf7ca10d1214ef2e3f9b171b68; 7bf0...,Medline; PMC,Study protocol for a randomised controlled fea...,10.1136/bmjopen-2020-041626,PMC7783524,33397667.0,cc-by-nc,INTRODUCTION: Patient adherence to adjuvant en...,2021-01-04,"Jacobs, Jamie M; Rapoport, Chelsea S; Horenste...",BMJ Open,NaN,NaN,NaN,document_parses/pdf_json/480c6748788911bf7ca10...,document_parses/pmc_json/PMC7783524.xml.json,https://doi.org/10.1136/bmjopen-2020-041626; h...,230508161.0,NaN,Up to 75% of female patients diagnosed with ea...
109,ctzu3vdc,96bf39fec5a29ffe6b3b7c674c772220b3c2741e; 3df4...,Elsevier; Medline; PMC; WHO,Telaprevir is a potential drug for repurposing...,10.1016/j.heliyon.2021.e07962,PMC8426143,34518806.0,no-cc,Drug repurposing is an important approach to t...,2021-09-09,"Mahmoud, Amal; Mostafa, Ahmed; Al-Karmalawy, A...",Heliyon,NaN,NaN,NaN,document_parses/pdf_json/96bf39fec5a29ffe6b3b7...,document_parses/pmc_json/PMC8426143.xml.json,https://api.elsevier.com/content/article/pii/S...,237444052.0,NaN,The coronavirus disease 2019 (COVID-19) pandem...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24957,krs3e945,d5e11b2e05e7d6d625975a20b09ad40e94808330; 986b...,PMC,Fast estimation of time-varying infectious dis...,10.1371/journal.pcbi.1008124,PMC7549798,32956345,cc-by,Compartmental epidemic models have been used e...,2020-09-21,"Jagan, Mikael; deJonge, Michelle S.; Krylova, ...",PLoS Comput Biol,NaN,NaN,NaN,document_parses/pdf_json/d5e11b2e05e7d6d625975...,document_parses/pmc_json/PMC7549798.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,NaN,The transmission rate of an infectious disease...
24958,et23099z,cb9ca2060dcc783d03a7fdfbba26ca5af8e01c08; 9b0b...,Elsevier; Medline; PMC,Multi-tier cloud infrastructure support for re...,10.1016/j.simpat.2016.06.005,PMC7270859,32518536.0,no-cc,The exceptional outbreaks of a number of epide...,2016-07-01,"Quwaider, Muhannad; Jararweh, Yaser",Simul Model Pract Theory,NaN,NaN,NaN,document_parses/pdf_json/cb9ca2060dcc783d03a7f...,document_parses/pmc_json/PMC7270859.xml.json,https://doi.org/10.1016/j.simpat.2016.06.005; ...,205175747.0,NaN,With the frequent spreading and outbreaks o

In [32]:
# There are ~13k rows where body_text_x is null but body_text_y is not null
df_merged.loc[df_merged.body_text_x.isnull() & df_merged.body_text_y.notnull()]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,body_text_x,body_text_y
40,j1sefamd,de4ddeb6cab0c5e13f3a6543ad5b7bb1215dc1da; 8e92...,Medline; PMC,The effect of eviction moratoria on the transm...,10.1038/s41467-021-22521-5,PMC8050248,33859196.0,cc-by,Massive unemployment during the COVID-19 pande...,2021-04-15,"Nande, Anjalika; Sheen, Justin; Walters, Emma ...",Nat Commun,NaN,NaN,NaN,document_parses/pdf_json/de4ddeb6cab0c5e13f3a6...,document_parses/pmc_json/PMC8050248.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/33859196/;...,233257667.0,NaN,The COVID-19 epidemic has caused unprecedented...
76,x1mb8so9,309cc336df9d42f9189c9e5b2221e177bdacc89b; 9604...,Medline; PMC,"Early Transmission Dynamics, Spread, and Genom...",10.3201/eid2702.203767,PMC7853578,33496228.0,no-cc,"We report an epidemiologic analysis of 4,210 c...",2021-02-19,"Franco, Danilo; Gonzalez, Claudia; Abrego, Ley...",Emerg Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/309cc336df9d42f9189c9...,document_parses/pmc_json/PMC7853578.xml.json,https://doi.org/10.3201/eid2702.203767; https:...,231711065.0,NaN,"To perform molecular detection of SARS-CoV-2, ..."
83,v9xo446h,08a3e8af143f9b97a9ba5220f0de0fe0ae1bba06; 15d4...,Elsevier; Medline; PMC,Regulation of stress granules in virus systems,10.1016/j.tim.2012.02.001,PMC3322245,22405519.0,green-oa,Virus infection initiates a number of cellular...,2012-04-01,"White, James P.; Lloyd, Richard E.",Trends in Microbiology,NaN,NaN,NaN,document_parses/pdf_json/08a3e8af143f9b97a9ba5...,document_parses/pmc_json/PMC3322245.xml.json,https://doi.org/10.1016/j.tim.2012.02.001; htt...,46260378.0,NaN,Eukaryotic cells can contain multiple types of...
86,39rck23u,480c6748788911bf7ca10d1214ef2e3f9b171b68; 7bf0...,Medline; PMC,Study protocol for a randomised controlled fea...,10.1136/bmjopen-2020-041626,PMC7783524,33397667.0,cc-by-nc,INTRODUCTION: Patient adherence to adjuvant en...,2021-01-04,"Jacobs, Jamie M; Rapoport, Chelsea S; Horenste...",BMJ Open,NaN,NaN,NaN,document_parses/pdf_json/480c6748788911bf7ca10...,document_parses/pmc_json/PMC7783524.xml.json,https://doi.org/10.1136/bmjopen-2020-041626; h...,230508161.0,NaN,Up to 75% of female patients diagnosed with ea...
109,ctzu3vdc,96bf39fec5a29ffe6b3b7c674c772220b3c2741e; 3df4...,Elsevier; Medline; PMC; WHO,Telaprevir is a potential drug for repurposing...,10.1016/j.heliyon.2021.e07962,PMC8426143,34518806.0,no-cc,Drug repurposing is an important approach to t...,2021-09-09,"Mahmoud, Amal; Mostafa, Ahmed; Al-Karmalawy, A...",Heliyon,NaN,NaN,NaN,document_parses/pdf_json/96bf39fec5a29ffe6b3b7...,document_parses/pmc_json/PMC8426143.xml.json,https://api.elsevier.com/content/article/pii/S...,237444052.0,NaN,The coronavirus disease 2019 (COVID-19) pandem...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24957,krs3e945,d5e11b2e05e7d6d625975a20b09ad40e94808330; 986b...,PMC,Fast estimation of time-varying infectious dis...,10.1371/journal.pcbi.1008124,PMC7549798,32956345,cc-by,Compartmental epidemic models have been used e...,2020-09-21,"Jagan, Mikael; deJonge, Michelle S.; Krylova, ...",PLoS Comput Biol,NaN,NaN,NaN,document_parses/pdf_json/d5e11b2e05e7d6d625975...,document_parses/pmc_json/PMC7549798.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN,NaN,The transmission rate of an infectious disease...
24958,et23099z,cb9ca2060dcc783d03a7fdfbba26ca5af8e01c08; 9b0b...,Elsevier; Medline; PMC,Multi-tier cloud infrastructure support for re...,10.1016/j.simpat.2016.06.005,PMC7270859,32518536.0,no-cc,The exceptional outbreaks of a number of epide...,2016-07-01,"Quwaider, Muhannad; Jararweh, Yaser",Simul Model Pract Theory,NaN,NaN,NaN,document_parses/pdf_json/cb9ca2060dcc783d03a7f...,document_parses/pmc_json/PMC7270859.xml.json,https://doi.org/10.1016/j.simpat.2016.06.005; ...,205175747.0,NaN,With the frequent spreading and outbreaks o

In [33]:
# We are trusting the text from pmc folder to be of higher quality as it contains full text. 
# Hence we will replace with body_text_x with body_text_y where body_text_y exists
df_merged.loc[df_merged.body_text_y.notnull(),'body_text_x'] = df_merged.loc[df_merged.body_text_y.notnull(), 'body_text_y']

In [34]:
# Lets get rid of the pdf_pmc body text column and rename the body text column
df_merged.rename(columns = {'body_text_x' : 'body_text'}, inplace = True)
df_merged.drop('body_text_y',axis=1,inplace = True)
df_merged.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id', 'body_text'],
      dtype='object')

In [35]:
# Body text null values have now decreased.
df_merged.body_text.isnull().sum()

88

In [36]:
df_merged.columns

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id', 'body_text'],
      dtype='object')

In [37]:
df_final = df_merged[['sha', 'title', 'abstract', 'publish_time', 'authors', 'url', 'body_text']]

In [38]:
df_final.head()

,sha,title,abstract,publish_time,authors,url,body_text
0,feedfe27a4eee49d8a1d09f50e8ecfe73057602a,Experiences and effects of telerehabilitation ...,BACKGROUND: The announcement of a national loc...,2021-06-30,"Ebrahim, Humairaa; Pillay-Jayaraman, Prithi; L...",https://www.ncbi.nlm.nih.gov/pubmed/34230898/;...,"As a result of the COVID-19 pandemic, many pat..."
1,c2ca01f12643a88e059e81619fb971fa61de3971,SCOAT-Net: A novel network for segmenting COVI...,Automatic segmentation of lung opacification f...,2021-06-10,"Zhao, Shixuan; Li, Zhidan; Chen, Yang; Zhao, W...",https://www.ncbi.nlm.nih.gov/pubmed/34127870/;...,"The coronavirus disease 2019 (COVID-19), which..."
2,d0d1e05bb14f068d8323807ba042860887b7aa00,A Comprehensive Review on Factors Influences B...,Exosomes are nanoscale-sized membrane vesicles...,2021-02-17,"Gurunathan, Sangiliyandi; Kang, Min-Hee; Kim, ...",https://doi.org/10.2147/ijn.s291956; https://w...,Extracellular vesicles (EVs) including exosome...
3,fb8c3a315b4a345dafc7e59be000e21bf965dea8,Opposition to cannabis legalization on public ...,NaN,2021-04-13,"Smyth, Bobby P; Christie, Grant IG",https://doi.org/10.1016/j.lanwpc.2021.100142; ...,Both authors contributed to the drafting of th...
4,ed4c180c272b047c6db958508fb9a8edd40cb1cb,An epigenetic signature to fight COVID-19,NaN,2021-05-13,"Herbein, Georges",https://www.ncbi.nlm.nih.gov/pubmed/33993054/;...,Research performed in our laboratories is supp...


In [39]:
df_final = df_final.dropna(axis=0,subset=['body_text', 'title'])
df_final.isnull().sum()

sha                0
title              0
abstract        2940
publish_time       0
authors          227
url                0
body_text          0
dtype: int64

In [40]:
df_final.shape

(24910, 7)

In [41]:
df_final.to_csv('FINAL_CORD_DATA.csv', index=False)